In [ ]:
!pip install yfinance numpy pandas tensorflow

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np

# Download historical stock data
ticker = 'ADANIENSOL.NS' 
data = yf.download(ticker)
prices = data['Close'].values

# Normalize the data
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
prices = scaler.fit_transform(prices.reshape(-1, 1))

# Prepare sequences for the TimeGAN model
def create_sequences(data, seq_length):
    sequences = []
    for i in range(len(data) - seq_length):
        sequences.append(data[i:i + seq_length])
    return np.array(sequences)

seq_length = 10 
sequences = create_sequences(prices, seq_length)

[*********************100%%**********************]  1 of 1 completed


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

class TimeGAN:
    def __init__(self, seq_length, feature_dim, hidden_dim, num_layers):
        self.seq_length = seq_length
        self.feature_dim = feature_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        # Build the TimeGAN model components
        self.build_generator()
        self.build_discriminator()
        self.build_supervisor()

    def build_generator(self):
        inputs = layers.Input(shape=(self.seq_length, self.feature_dim))
        x = inputs
        for _ in range(self.num_layers):
            x = layers.LSTM(self.hidden_dim, return_sequences=True)(x)
        outputs = layers.TimeDistributed(layers.Dense(self.feature_dim))(x)
        self.generator = tf.keras.models.Model(inputs, outputs, name='Generator')

    def build_discriminator(self):
        inputs = layers.Input(shape=(self.seq_length, self.feature_dim))
        x = inputs
        for _ in range(self.num_layers):
            x = layers.LSTM(self.hidden_dim, return_sequences=True)(x)
        outputs = layers.TimeDistributed(layers.Dense(1, activation='sigmoid'))(x)
        self.discriminator = tf.keras.models.Model(inputs, outputs, name='Discriminator')

    def build_supervisor(self):
        inputs = layers.Input(shape=(self.seq_length, self.feature_dim))
        x = inputs
        for _ in range(self.num_layers):
            x = layers.LSTM(self.hidden_dim, return_sequences=True)(x)
        outputs = layers.TimeDistributed(layers.Dense(self.feature_dim))(x)
        self.supervisor = tf.keras.models.Model(inputs, outputs, name='Supervisor')

    def compile(self, optimizer):
        self.generator.compile(loss='mse', optimizer=optimizer, metrics=[tf.keras.metrics.MeanAbsoluteError()])
        self.discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
        self.supervisor.compile(loss='mse', optimizer=optimizer, metrics=[tf.keras.metrics.MeanAbsoluteError()])

    def train(self, sequences, batch_size, epochs):
        for epoch in range(epochs):
            idx = np.random.randint(0, sequences.shape[0], batch_size)
            real_sequences = sequences[idx]

            noise = np.random.normal(0, 1, (batch_size, self.seq_length, self.feature_dim))
            fake_sequences = self.generator.predict(noise)

            d_loss_real, d_acc_real = self.discriminator.train_on_batch(real_sequences, np.ones((batch_size, self.seq_length, 1)))
            d_loss_fake, d_acc_fake = self.discriminator.train_on_batch(fake_sequences, np.zeros((batch_size, self.seq_length, 1)))

            # Train the generator
            g_loss, g_mae = self.generator.train_on_batch(noise, real_sequences)

            # Train the supervisor
            s_loss, s_mae = self.supervisor.train_on_batch(noise, real_sequences)

            if epoch % 100 == 0:
                print(f"Epoch {epoch} / {epochs-1} - D loss: {(d_loss_real + d_loss_fake) / 2}, D acc: {(d_acc_real + d_acc_fake) / 2}, G loss: {g_loss}, G MAE: {g_mae}, S loss: {s_loss}, S MAE: {s_mae}")


# Hyperparameters
seq_length = 10
feature_dim = 1
hidden_dim = 24
num_layers = 3
batch_size = 32
epochs = 1001
learning_rate = 0.001

# Initialize and compile the TimeGAN model
timegan = TimeGAN(seq_length, feature_dim, hidden_dim, num_layers)
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate)
timegan.compile(optimizer)

In [ ]:
timegan.train(sequences, batch_size, epochs)

1/1 [==============================] - 2s 2s/step
Epoch 0 / 1000 - D loss: 0.6915155649185181, D acc: 0.5, G loss: 0.3205198645591736, G MAE: 0.5056183934211731, S loss: 0.3198513090610504, S MAE: 0.5054426193237305
1/1 [==============================] - 0s 22ms/step
Epoch 100 / 1000 - D loss: 0.5228853970766068, D acc: 0.582812488079071, G loss: 0.06445319950580597, G MAE: 0.21426498889923096, S loss: 0.06579272449016571, S MAE: 0.21730630099773407
1/1 [==============================] - 0s 27ms/step
Epoch 200 / 1000 - D loss: 0.26500581949949265, D acc: 0.893750011920929, G loss: 0.07382099330425262, G MAE: 0.2288438379764557, S loss: 0.07343176752328873, S MAE: 0.22782108187675476
1/1 [==============================] - 0s 24ms/step
Epoch 300 / 1000 - D loss: 0.1986505091190338, D acc: 0.942187488079071, G loss: 0.07225728034973145, G MAE: 0.24117819964885712, S loss: 0.07213981449604034, S MAE: 0.24028821289539337
1/1 [==============================] - 0s 25ms/step
Epoch 400 / 1000 -

In [ ]:
# Generate predictions for the next 1 day and 1 week
def predict_next_steps(model, data, steps):
    predictions = []
    input_seq = data[-seq_length:].reshape(1, seq_length, feature_dim)
    for _ in range(steps):
        next_step = model.generator.predict(input_seq)
        predictions.append(next_step[:, -1, :])
        input_seq = np.append(input_seq[:, 1:, :], next_step[:, -1:, :], axis=1)
    return np.array(predictions).reshape(-1, feature_dim)

# Predict the next day and week prices
next_1d_pred = predict_next_steps(timegan, prices, 1)
next_1w_pred = predict_next_steps(timegan, prices, 7)

# Transform the predictions back to the original scale
next_1d_pred = scaler.inverse_transform(next_1d_pred)
next_1w_pred = scaler.inverse_transform(next_1w_pred)

print(f'1-Day Forecast: {next_1d_pred}')
print(f'1-Week Forecast: {next_1w_pred}')

1/1 [==============================] - 0s 24ms/step
1-Day Forecast: [[968.3561]]
1-Week Forecast: [[968.3561 ]
 [968.2799 ]
 [968.45154]
 [968.6657 ]
 [968.8461 ]
 [968.9799 ]
 [969.0788 ]]
